# File Connector

The FileConnector gives agents tools to operate on files: create, read, edit, search, and delete. All operations go through the workspace sandbox, so the agent sees `/workspace/...` paths while files live in isolated host directories.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.connectors.file import FileConnector
from agentic_patterns.core.user_session import set_user_session

## Setup and Wrap FileConnector as Agent Tools

Set the user/session context once, then wrap FileConnector methods as agent tools. No need to pass context through -- it's resolved automatically from contextvars.

In [ ]:
set_user_session("demo", "file_connector_demo")

connector = FileConnector()

tools = [
    connector.append,
    connector.delete,
    connector.edit,
    connector.find,
    connector.head,
    connector.list,
    connector.read,
    connector.write,
]

## Agent Creates and Edits a File

We ask the agent to create a markdown file, then modify it. The agent decides which tools to call and in what order.

In [ ]:
agent = get_agent(tools=tools)

prompt = """Do the following steps:
1. Create a file /workspace/notes.md with a title '# Meeting Notes' and three bullet points about a project kickoff meeting.
2. Read the file back to verify it was created.
3. Edit line 3 to change the second bullet point to '- Agreed on weekly sync every Monday'.
4. Read the file again and show me the final content."""

agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\nFinal answer: {agent_run.result.output}")

## Verify on Disk

The file exists in the host filesystem under the user's workspace directory.

In [ ]:
from pathlib import PurePosixPath
from agentic_patterns.core.workspace import workspace_to_host_path

host_path = workspace_to_host_path(PurePosixPath("/workspace/notes.md"))
print(f"Host path: {host_path}")
print(f"Content:\n{host_path.read_text()}")